In [1]:
from typing import Optional
from pennylane import numpy as np
import pennylane as qml

In [2]:
from qml_essentials.ansaetze import Ansaetze

all_ansaetze = Ansaetze.get_available()

for ansatz in all_ansaetze:
    print(ansatz.__name__)

No_Ansatz
Circuit_1
Circuit_2
Circuit_3
Circuit_4
Circuit_6
Circuit_9
Circuit_10
Circuit_15
Circuit_16
Circuit_17
Circuit_18
Circuit_19
No_Entangling
Strongly_Entangling
Hardware_Efficient


In [3]:
from qml_essentials.ansaetze import Circuit


class MyHardwareEfficient(Circuit):
    @staticmethod
    def n_params_per_layer(n_qubits: int) -> int:
        return n_qubits * 3

    @staticmethod
    def get_control_indices(n_qubits: int) -> Optional[np.ndarray]:
        return None

    @staticmethod
    def build(w: np.ndarray, n_qubits: int, noise_params=None):
        w_idx = 0
        for q in range(n_qubits):
            qml.RY(w[w_idx], wires=q)
            w_idx += 1
            qml.RZ(w[w_idx], wires=q)
            w_idx += 1

        if n_qubits > 1:
            for q in range(n_qubits - 1):
                qml.CZ(wires=[q, q + 1])


from qml_essentials.model import Model

model = Model(
    n_qubits=2,
    n_layers=1,
    circuit_type=MyHardwareEfficient,
)

In [4]:
from qml_essentials.model import Model
from qml_essentials.ansaetze import Gates


def MyCustomEncoding(w, wires, noise_params=None):
    Gates.RX(w[0], wires, noise_params=noise_params)
    Gates.RY(w[1], wires, noise_params=noise_params)


model = Model(
    n_qubits=2,
    n_layers=1,
    circuit_type=MyHardwareEfficient,
    encoding=MyCustomEncoding,
)

model(inputs=[1, 2])

tensor([ 0.42649347, -0.5008437 ], requires_grad=True)

In [ ]:
noise_params = {
    "BitFlip": 0.0,
    "PhaseFlip": 0.0,
    "AmplitudeDamping": 0.0,
    "PhaseDamping": 0.0,
    "Depolarizing": 0.0,
    "MultiQubitDepolarizing": 0.0,
}


class MyNoisyHardwareEfficient(Circuit):
    @staticmethod
    def n_params_per_layer(n_qubits: int) -> int:
        return n_qubits * 3

    @staticmethod
    def get_control_indices(n_qubits: int) -> Optional[np.ndarray]:
        return None

    @staticmethod
    def build(w: np.ndarray, n_qubits: int, noise_params=None):
        w_idx = 0
        for q in range(n_qubits):
            Gates.RY(w[w_idx], wires=q, noise_params=noise_params)
            w_idx += 1
            Gates.RZ(w[w_idx], wires=q, noise_params=noise_params)
            w_idx += 1

        if n_qubits > 1:
            for q in range(n_qubits - 1):
                Gates.CZ(wires=[q, q + 1], noise_params=noise_params)


model(
    model.params,
    inputs=None,
    execution_type="density",
    noise_params={
        "BitFlip": 0.01,
        "PhaseFlip": 0.02,
        "AmplitudeDamping": 0.03,
        "PhaseDamping": 0.04,
        "Depolarizing": 0.05,
        "MultiQubitDepolarizing": 0.06,
    },
)

tensor([[ 6.32470553e-01-8.32667268e-19j,
          3.87151515e-02+1.17521966e-01j,
         -4.94760255e-02-4.37715289e-01j,
         -8.57727031e-02+2.67342370e-02j],
        [ 3.87151515e-02-1.17521966e-01j,
          2.69463691e-02+0.00000000e+00j,
         -8.74075510e-02-1.80601774e-02j,
         -2.65040244e-04+1.89353842e-02j],
        [-4.94760255e-02+4.37715289e-01j,
         -8.74075510e-02+1.80601774e-02j,
          3.26492035e-01-2.69229083e-17j,
         -1.25268413e-02-6.52209442e-02j],
        [-8.57727031e-02-2.67342370e-02j,
         -2.65040244e-04-1.89353842e-02j,
         -1.25268413e-02+6.52209442e-02j,
          1.40910425e-02+0.00000000e+00j]], requires_grad=True)